In [3]:
# Data handling and processing
import pandas as pd
import numpy as np
import math


# Sklearn imports
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import RFE
from sklearn import metrics

# Model imports
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# File mgmt
import os

os.chdir('Data')
!ls

Transactions_Village114
output.xlsx
output2.xlsx


In [4]:
path = 'Transactions_Village114/ds3170_tx_All_Data_5208_2019_1211_082948.txt'
df = pd.read_csv(path, sep="\t")
cols = ['Anon Student Id', 'Session Id', 'Duration (sec)', 'Level (Tutor Name)', 'Level (Tutor)', 'Problem Name', 'Problem View','Step Name', 'Attempt At Step','Is Last Attempt','Outcome', 'Input','CF (File)','CF (Matrix)','CF (Matrix Level)', 'CF (Matrix Order)', 'CF (Total Activity Problems)']
df = df[cols]
bubble_pop_df = df[df['Level (Tutor Name)'] == 'bubble_pop']
bpop_math_df = bubble_pop_df[bubble_pop_df['CF (Matrix)'] == 'math'] #6172

# X and y -> inputs and outputs for the classification model
cols = ['Duration (sec)', 'Level (Tutor)', 'Attempt At Step', 'CF (Matrix Level)', 'CF (Matrix Order)', 'Outcome']
X = bpop_math_df[cols]
y = X[['Outcome']]
X = X.drop(['Outcome'], axis=1)

C:\Users\jithe\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,7,37,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Helpful functions

In [9]:
# Clean duration (sec) columns: Convert all values to float, if not able to convert set value in that row as duration_mean
def clean_duration_col_with_mean(X):
    strings = 0
    ints = 0
    floats = 0
    other = 0
    cnt = 0
    sums = 0
    posn = []
    X_duration_arr = X['Duration (sec)'].array

    for i in range(len(X_duration_arr)):
        if isinstance(X_duration_arr[i], int):
            ints += 1
        elif isinstance(X_duration_arr[i], str):
            strings += 1
            try:
                X_duration_arr[i] = float(X_duration_arr[i])
            except ValueError:
#                 print("Cant convert to a float", X_duration_arr[i], "Position: ",i)
                posn.append(i)
        elif isinstance(X_duration_arr[i], float):
            floats += 1
    
        if isinstance(X_duration_arr[i], float):
            sums += X_duration_arr[i]
            cnt += 1

    duration_mean = sums/cnt
    for i in range(len(posn)):
        X_duration_arr[posn[i]] = duration_mean

    for i in range(len(X_duration_arr)):
        col = bpop_math_df.columns.get_loc('Duration (sec)')
        val = X_duration_arr[i]
        X.iloc[i, col] = val
    
    return X

# Ordinal encodes a column
def ordinal_encode_col(X, colname):
    enc = OrdinalEncoder()
    level_tutor = X[colname].tolist()
    tutor_level_2dlist = []
    for i in range(len(level_tutor)):
        tutor_level_2dlist.append([level_tutor[i]])

    enc.fit(tutor_level_2dlist)

    col = X.columns.get_loc(colname)
    num_entries = X.count()[colname]
    for i in range(num_entries):
        val = X.iloc[i, col]
        val = enc.transform([[val]])[0][0]
        X.iloc[i, col] = val
    
    return X

def rfe(X_train, y_train, apply_rfe):
    if apply_rfe == False:
        return
    data_final_vars=X_train.columns.values.tolist()
    y_=['y']
    X=[i for i in data_final_vars if i not in y_]
    logreg = LogisticRegression()
    rfe = RFE(logreg, 20)
    rfe = rfe.fit(X_train, y_train)
#     print("RFE Support: ",rfe.support_)
#     print("RFE Ranking: ", rfe.ranking_)

# incorrect -> 0, correct -> 1
# set_zero = "INCORRECT" means incorrect will be set to 0 and other value as 1
def encode_outputs(y, colname, set_zero):
    col = y.columns.get_loc(colname)
    for i in range(y.count()[colname]): 
        val = y.iloc[i, col]
        if val == set_zero:
            val = 0
        else: 
            val = 1
        y.iloc[i, col] = val
    
    return y

def encode_categorical_cols(X, encoder, col):

#   Ordinally encode categorical columns
    if encoder == "ordinal":
        for i in range(len(col)):
            X = ordinal_encode_col(X, col[i])
    
    return X

def normalize_data(X, norm):
    if norm == 'l1':
        X = sklearn.preprocessing.normalize(X, norm='l1', axis=1, copy=True, return_norm=False)
        X = pd.DataFrame(data=X, columns=cols[:len(cols)-1])
    
    elif norm == 'l2':
        X = sklearn.preprocessing.normalize(X, norm='l2', axis=1, copy=True, return_norm=False)
        X = pd.DataFrame(data=X, columns=cols[:len(cols)-1])
    
    return X


def run_logistic_regression(X, y, test_split, apply_rfe):
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)
    rfe(X_train, y_train, apply_rfe)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    performance = 100 * logreg.score(X_test, y_test)
#     print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(performance))
    return performance
    
def run_k_neighbors_classifier(X, y, test_split, apply_rfe, neighbors):
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)
    rfe(X_train, y_train, apply_rfe)       
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(X_train, y_train)
    performance = 100 * neigh.score(X_test, y_test)
#     print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(performance))
    return performance

def run_svm(X, y, test_split, apply_rfe, svc_kernel):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split)
    rfe(X_train, y_train, apply_rfe)   
    svm_model = svm.SVC(kernel=svc_kernel)
    print("CREATE MODEL")
    svm_model.fit(X_train, y_train)
    print("FIT")
    #Predict the response for test dataset
    y_pred = svm_model.predict(X_test)
    perf = 100 * metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:",perf)
    return perf

# def run_nn(X, y test_split):
    
    
    
def run_algo(algo, X, y, clean_method, encoder, norm, test_split, apply_rfe, categorical_cols, neighbors, svc_kernel):
    
    if clean_method == "mean":
        X = clean_duration_col_with_mean(X)
    
    X = encode_categorical_cols(X, encoder, categorical_cols)
    X = normalize_data(X, norm)
    y = encode_outputs(y, 'Outcome', 'INCORRECT').values.ravel()
    perf = 0
    if algo == "logistic_regression":
        perf = run_logistic_regression(X, y, test_split, apply_rfe)
    
    elif algo == "k_neighbors_classifier":
        perf = run_k_neighbors_classifier(X, y, test_split, apply_rfe, neighbors)
    
    elif algo == "svm":
        perf = run_svm(X, y, test_split, apply_rfe, svc_kernel)
        
    elif algo == "nn":
        perf = run_nn(X, y, test_split)
      
    return perf
    

In [ ]:
# mean median or mode (#FIXME: only supports mean for now)
clean_method = "mean" 

# ordinal, onehot etc.. (#FIXME: only supports ordinal encoding for now)
encoder = "ordinal" 

# cols that are categories rather than numbers and have to be encoded
categorical_cols = ['Level (Tutor)']

# none, l1, l2 etc..
norm = "none" 

# set True to apply Recursive Feature Elimination
apply_rfe = True

test_split = 0.2
num_runs = 10

# k neighbors classifier
neighbors = 5

# linear, poly, rbf, sigmoid
svc_kernel = 'linear'

# Supports logistic_regression, k_neighbors_classifier, svm, nn
algo = "nn"

# used to store performance of each run in num_runs
total_perfs = []

import time
start = time.time()

for i in range(num_runs):
    X = bpop_math_df[cols]
    X = X.sample(frac=1).reset_index(drop=True)
    y = X[['Outcome']]
    X = X.drop(['Outcome'], axis=1)
    
    perf = run_algo(algo, X, y, clean_method, encoder, norm, test_split, apply_rfe, categorical_cols, neighbors, svc_kernel)
    total_perfs.append(perf)

end = time.time()
print(end - start)

avg_perf = sum(total_perfs)/num_runs
print("Avg. performance over ", num_runs, "runs: ", avg_perf)

CREATE MODEL


In [ ]:
print(avg_perf, "%(", min(total_perfs), "-", max(total_perfs) ,")")

In [1]:
import torch 
import torch.nn as nn

NameError: name 'X' is not defined

In [6]:
class StudentNet(nn.Module):
    
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        
        super(StudentNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
    
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

model = StudentNet(5, 10, 10, 2)
print(model)

,Outcome
0,INCORRECT
1,CORRECT
2,INCORRECT
3,INCORRECT
4,INCORRECT
...,...
166060,CORRECT
166061,INCORRECT
166062,INCORRECT
166063,CORRECT


In [7]:
X

,Duration (sec),Level (Tutor),Attempt At Step,CF (Matrix Level),CF (Matrix Order)
0,0,bpop.num_0..9.by.1.asc.q2q.AV.rise.12__it_2,1,1,30
1,.,bpop.num_0..9.by.1.asc.q2q.AV.rise.12__it_2,2,1,30
2,122,bpop.num_0..9.by.1.asc.q2q.AV.rise.12__it_2,1,1,30
3,66,bpop.num_0..9.by.1.asc.q2q.AV.rise.12__it_2,2,1,30
4,97,bpop.num_0..9.by.1.asc.q2q.AV.rise.12__it_2,3,1,30
...,...,...,...,...,...
166060,6,bpop.mn_4..9.MN-SD-UP-OFF1-BL2.decr.19,1,4,112
166061,9,bpop.mn_4..9.MN-SD-UP-OFF1-BL2.decr.19,1,4,112
166062,12,bpop.mn_4..9.MN-SD-UP-OFF1-BL2.decr.19,2,4,112
166063,12,bpop.mn_4..9.MN-SD-UP-OFF1-BL2.decr.19,3,4,112


In [12]:
X = clean_duration_col_with_mean(X)
X = encode_categorical_cols(X, 'ordinal', ['Level (Tutor)'])
y = encode_outputs(y, 'Outcome', 'INCORRECT').values.ravel()

In [14]:
batch_size = 60
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train_loader = torch.utils.data.DataLoader(dataset=X_train, batch_size=batch_size, shuffle=True)
X_test_loader = torch.utils.data.DataLoader(dataset=X_test, batch_size=batch_size, shuffle=True)

y_train_loader = torch.utils.data.DataLoader(dataset=y_train, batch_size=batch_size, shuffle=True)
y_test_loader = torch.utils.data.DataLoader(dataset=y_test, batch_size=batch_size, shuffle=True)

y_train_loader

In [17]:
X_train['Outcome'] = y_train

C:\Users\jithe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
y_train

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [18]:
trainset = X_train
testset = X_test

X_train['Outcome'] = y_train
X_yest['Outcome'] = y_test

,Duration (sec),Level (Tutor),Attempt At Step,CF (Matrix Level),CF (Matrix Order),Outcome
12355,6,29,6.0,9,291,0
41803,7,12,7.0,4,104,1
123820,5,10,5.0,4,107,0
142498,8,80,8.0,2,74,1
82911,5,37,5.0,4,109,0
...,...,...,...,...,...,...
82309,7,36,7.0,4,110,1
56273,7,36,7.0,4,110,1
12018,12,44,12.0,9,287,0
42336,16,13,16.0,4,103,1
